In [108]:
import requests
from bs4 import BeautifulSoup, SoupStrainer 
import sys
import string
import os
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.cross_validation import train_test_split, KFold

In [10]:
url = 'http://homeopathicremediesandtreatment.com/'

In [8]:
print len('http://')#homeopathicremediesandtreatment.com/')

7


In [37]:
#Function takes website request text and scrapes all links on that site
#Not functioning properly yet
def Scrape_Whole_Site(url):
    link_list = []
    master_list = []
    successes = 0
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    master_links = soup.find_all('a')
    
    for link in master_links:
        link_list.append(link['href'])
        master_list.append(link['href'])
    
    #for link in link_list:
        #print link
        #print link[:len(url)]
    
    for link in link_list:
        try:
            sub_request = requests.get(link)
        except:
            pass
        sub_soup = BeautifulSoup(sub_request.text, 'html.parser')
        sub_links = sub_soup.find_all('a')
        for sub_link in sub_links:
            try:
                if sub_link['href'][:len(url)] == url:
                    if sub_link['href'] not in master_list:
                        #print sub_link['href']
                        master_list.append(sub_link['href'])
                        successes += 1
                        sys.stdout.write("\r" + 'Sucesses: ' + str(successes))   
            except:
                pass
        
    first_len = len(master_list)
    set_list = set(master_list)
    set_len = len(master_list)
    
    return first_len, set_len
    
first_len, set_len = Scrape_Whole_Site(url)
print ' '
print 'Length: ' + str(first_len)
print 'Set len: ' + str(set_len)
print 'Done'
for url in master_list:
    print url

Sucesses: 1682 
Length: 1724
Set len: 1724
Done
http://homeopathicremediesandtreatment.com/Homeopathic-Remedies-and-Treatment.php
http://homeopathicremediesandtreatment.com/Hahnemann-Homeopathy.php
http://homeopathicremediesandtreatment.com/Gemmotherapy-What-is-Gemmotherapy.php
http://homeopathicremediesandtreatment.com/Oligotherapy-Oligo-Trace-Elements.php
http://homeopathicremediesandtreatment.com/Biochemic-Tissue-Salts-Schuessler-Cell-Salts.php
http://homeopathicremediesandtreatment.com/Organotherapy-What-is-Organotherapy.php
http://homeopathicremediesandtreatment.com/Liver-Detoxification--Homeopathy.php
http://homeopathicremediesandtreatment.com/Eczema-Skin-Eruption-Suppression.php
http://homeopathicremediesandtreatment.com/Homeopathic-Case-Taking-Questionnaire-Homeopathy.php
http://homeopathicremediesandtreatment.com/Acid-base-Balance-Body-pH-Balance.php
http://homeopathicremediesandtreatment.com/10-Food-Combination-Rules.php
http://homeopathicremediesandtreatment.com/Arthritis-Ca

In [41]:
letters = [string.ascii_lowercase]
letters

['abcdefghijklmnopqrstuvwxyz']

In [48]:
#http://www.homeopathycenter.org/   #doesn't take many requests to jam up server
url = 'http://www.homeopathycenter.org/find-remedy/a'# + letters[0]
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
text = soup.find_all('p')
soup.text

ConnectionError: HTTPConnectionPool(host='www.homeopathycenter.org', port=80): Max retries exceeded with url: /find-remedy/a (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x1081cb690>: Failed to establish a new connection: [Errno 60] Operation timed out',))

###Assumed kook source

In [154]:
#http://www.homeoint.org    #Scrapes Homeopathy site
for i in range(40):
    url = 'http://www.homeoint.org/hompath/articles/' + str(i) + '.html'
    r = requests.get(url)
    if r.status_code != 404:
        try:
            soup = BeautifulSoup(r.text, 'html.parser')
            titles = soup.find_all('title')
            the_title = titles[0].text
            text = soup.find_all('br')
            content = text[0].text
            with open('/users/markregalla/desktop/metis/Project4/HomeoArticles/page_' 
                      + str(i) +'.txt', 'w') as myfile:
                #myfile.write((the_title + '\n' + content).encode("UTF-8"))
                myfile.write((the_title + content).encode("UTF-8"))
            myfile.close
        except:
            pass
    sys.stdout.write("\r" + str(i))   

39

###Assumed genuine source

In [135]:
#Mayo Clinic Drugs and supplements articles
url = 'http://www.mayoclinic.org/drugs-supplements'
r = requests.get(url)
print r.status_code
soup = BeautifulSoup(r.text, 'html.parser')
i = 0
ref_list = []
for link in soup.find_all('a'):
    if i in range(108, 149):
        ref_list.append(link['href'])
    i += 1
k = 0
for link in ref_list:
    r = requests.get('http://www.mayoclinic.org/' + link)
    soup = BeautifulSoup(r.text, 'html.parser')
    texts = soup.find_all('p')
    text_list = []
    i = 0 
    for text in texts:
        text_list.append(text.text)
        i += 1
    
    document = ''
    for element in text_list[3:len(texts) - 8]:
        document = document + str(element.encode("UTF-8"))
    try:
        with open('/users/markregalla/desktop/metis/Project4/MayoArticles/page_' 
                          + str(k) +'.txt', 'w') as myfile:
            myfile.write(document.encode("UTF-8"))
        myfile.close
    except:
        pass
    sys.stdout.write("\r" + str(k)) 
    k += 1

200
40

###Run scraped articles through Naive Bayes classifier

In [19]:
#Mayo list
Mayo_list = []
for article in os.listdir('/users/markregalla/desktop/metis/Project4/MayoArticles'):
    with open("/users/markregalla/desktop/metis/Project4/MayoArticles/" 
              + article, "r") as myfile:
        stuff = myfile.read()
    myfile.close
    if stuff != '':
        Mayo_list.append(stuff)
print len(Mayo_list)
#Mayo_list

35


In [68]:
#Homeopathic list
Homeo_list = []
k = 0
for article in os.listdir('/users/markregalla/desktop/metis/Project4/HomeoArticles'):
    with open('/users/markregalla/desktop/metis/Project4/HomeoArticles/' 
              + article) as myfile:
        stuff = myfile.read()
    myfile.close
    if stuff != '':
        Homeo_list.append(str(stuff.decode("ascii", "ignore")))
    k += 1
    if k == 35:
        break
#print Homeo_list
'''
total_list = []
for word in Homeo_list[0]:
    total_list.append(word.decode("ascii", "ignore"))
print total_list[0]
'''

'\ntotal_list = []\nfor word in Homeo_list[0]:\n    total_list.append(word.decode("ascii", "ignore"))\nprint total_list[0]\n'

In [100]:
Mayo_word_token = []
Mayo_sent_token = []
Homeo_word_token = []
Homeo_sent_token = []

for word in Mayo_list:
    Mayo_word_token.append(word_tokenize(word))
    Mayo_sent_token.append(sent_tokenize(word))
for word in Homeo_list:
    Homeo_word_token.append(word_tokenize(word))
    Homeo_sent_token.append(sent_tokenize(word))

In [102]:
all_articles = []
all_labels = []
for i in range(len(Homeo_word_token)):
    Homeo_word_token[i]=str(Homeo_word_token[i]).replace("[","")
    Homeo_word_token[i]=str(Homeo_word_token[i]).replace("]","")
    all_articles.append(Homeo_word_token[i])
    all_labels.append('Homeo')
    Mayo_word_token[i]=str(Mayo_word_token[i]).replace("[","")
    Mayo_word_token[i]=str(Mayo_word_token[i]).replace("]","")
    all_articles.append(Mayo_word_token[i])
    all_labels.append('Mayo')

data = pd.DataFrame(columns=['Name','Text'])
data.Name=all_labels
data.Text=all_articles

In [152]:
vectorizer = CountVectorizer()  #ngram_range=(1,2))
vectorizer.fit(data.Text)
x = vectorizer.transform(data.Text)
data.head(10)

,Name,Text
0,Homeo,"'Tresorie', '-', 'Application', 'of', 'homoeop..."
1,Mayo,"'Lactobacillus', 'acidophilus', 'is', 'a', 'me..."
2,Homeo,"'Tresorie', '-', 'The', 'Homoeopathic', 'Aspec..."
3,Mayo,"'Clear', 'gel', 'from', 'the', 'aloe', 'leaves..."
4,Homeo,"'Tresorie', '-', 'A', 'Resume', 'of', 'Appendi..."
5,Mayo,"'Creatine', 'is', 'normally', 'found', 'in', '..."
6,Homeo,"'Tresorie', '-', 'OBSERVATIONS', 'ON', 'SOME',..."
7,Mayo,"'Dehydroepiandrosterone', '(', 'DHEA', ')', 'i..."
8,Homeo,"'Tresorie', '-', 'SURGICAL', 'SHOCK', '', 'Sur..."
9,Mayo,"'Dong', 'quai', 'is', 'also', 'known', 'as', '..."


In [153]:
y = data.Name
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
#print x_train, x_test, y_train, y_test

<class 'pandas.core.series.Series'>


In [140]:
print type(y_test)
y_test = y_test.reset_index()
print type(y_test)
y_test

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


,index,Name
0,22,Homeo
1,0,Homeo
2,49,Mayo
3,4,Homeo
4,54,Homeo
5,18,Homeo
6,10,Homeo
7,33,Mayo
8,45,Mayo
9,12,Homeo


In [141]:
model = MultinomialNB().fit(x_train, y_train)
results = model.predict(x_test)
print type(results)
for i in range(len(results)):
    print results[i], y_test.Name[i]

<type 'numpy.ndarray'>
Homeo Homeo
Homeo Homeo
Mayo Mayo
Homeo Homeo
Homeo Homeo
Homeo Homeo
Homeo Homeo
Mayo Mayo
Mayo Mayo
Homeo Homeo
Mayo Mayo
Mayo Mayo
Mayo Mayo
Mayo Mayo
Homeo Homeo
Homeo Homeo
Homeo Homeo
Mayo Mayo


In [149]:
#X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
#y = np.array([1, 2, 3, 4])
kf = KFold(35, n_folds=7)
#print len(kf)
print type(x), type(y)

#print(kf)  
for train_index, test_index in kf:
    #print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    cross_model = MultinomialNB().fit(x_train, y_train)
    results = model.predict(x_test)
    #print type(results)
    print results
    #print y_test.transpose()
    print list(y_test)
    #for i in range(len(results)):
    #    print results[i], y_test[i]

<class 'scipy.sparse.csr.csr_matrix'> <class 'pandas.core.series.Series'>
['Homeo' 'Mayo' 'Homeo' 'Mayo' 'Homeo']
['Homeo', 'Mayo', 'Homeo', 'Mayo', 'Homeo']
['Mayo' 'Homeo' 'Mayo' 'Homeo' 'Mayo']
['Mayo', 'Homeo', 'Mayo', 'Homeo', 'Mayo']
['Homeo' 'Mayo' 'Homeo' 'Mayo' 'Homeo']
['Homeo', 'Mayo', 'Homeo', 'Mayo', 'Homeo']
['Mayo' 'Homeo' 'Mayo' 'Homeo' 'Mayo']
['Mayo', 'Homeo', 'Mayo', 'Homeo', 'Mayo']
['Homeo' 'Mayo' 'Homeo' 'Mayo' 'Homeo']
['Homeo', 'Mayo', 'Homeo', 'Mayo', 'Homeo']
['Mayo' 'Homeo' 'Mayo' 'Homeo' 'Mayo']
['Mayo', 'Homeo', 'Mayo', 'Homeo', 'Mayo']
['Homeo' 'Mayo' 'Homeo' 'Mayo' 'Homeo']
['Homeo', 'Mayo', 'Homeo', 'Mayo', 'Homeo']
